# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("output_data/cities.csv")
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,54.10,90,75,4.85,AU,1666735745
1,Belgrade,44.8040,20.4651,54.43,100,0,2.30,RS,1666735610
2,Ribeira Grande,38.5167,-28.7000,61.77,74,85,3.00,PT,1666735615
3,Mareeba,-17.0000,145.4333,80.47,83,0,3.94,AU,1666735583
4,Erenhot,43.6475,111.9767,32.90,66,8,11.14,CN,1666735746


In [3]:
city_weather_df.count()

City          527
Lat           527
Lng           527
Max Temp      527
Humidity      527
Cloudiness    527
Wind Speed    527
Country       525
Date          527
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)
lat_lng = city_weather_df[['Lat','Lng']]
humidity = city_weather_df['Humidity'].astype(float)


In [5]:
#create the heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_lng,weights=humidity,dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#weather conditions:
#1.max temp < 80 but > 70
#2.wind speed < 10 mph
#3.zero cloudiness
#drop any rows that do not satisfy all three conditions

perf_temp = city_weather_df.loc[(city_weather_df['Max Temp'] < 80)& 
                                (city_weather_df['Max Temp'] > 70)&
                                (city_weather_df['Wind Speed'] < 10)&
                                (city_weather_df['Cloudiness'] == 0)]

perf_temp.count()

City          17
Lat           17
Lng           17
Max Temp      17
Humidity      17
Cloudiness    17
Wind Speed    17
Country       17
Date          17
dtype: int64

In [7]:
#drop rows with null values
perf_temp = perf_temp.dropna(how='any')
perf_temp.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Nālūt,30.3333,10.8500,70.11,26,0,7.27,LY,1666735747
28,Kiryat Gat,31.6100,34.7642,77.68,40,0,6.31,IL,1666735753
74,San Patricio,28.0170,-97.5169,78.75,20,0,8.97,US,1666735766
91,Dwārka,22.2394,68.9678,79.45,62,0,9.73,IN,1666735771
99,Nāwa,27.0167,75.0000,71.22,33,0,6.80,IN,1666735774


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#add a hotel name column 
perf_temp['Hotel Name'] = ''


In [9]:
perf_temp.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Nālūt,30.3333,10.8500,70.11,26,0,7.27,LY,1666735747,
28,Kiryat Gat,31.6100,34.7642,77.68,40,0,6.31,IL,1666735753,
74,San Patricio,28.0170,-97.5169,78.75,20,0,8.97,US,1666735766,
91,Dwārka,22.2394,68.9678,79.45,62,0,9.73,IN,1666735771,
99,Nāwa,27.0167,75.0000,71.22,33,0,6.80,IN,1666735774,


In [10]:
#smaller sample size to avoid overuse of the Google Maps API call limits

# perf_temp_reduced = perf_temp.iloc[0:10,:]
# perf_temp_reduced

In [11]:
##for each city, use Google Places API to find first hotel located within
#5,000 meters of the coordinates

#look at airport ratings lesson for instruction, and use iterrows
# set up a parameters dictionary
params = {
    "radius": 5000,
    "types": 'lodging',
    "keyword": "Hotel",
    "key": g_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in perf_temp.iterrows():
    lat= row['Lat']
    lng= row['Lng']
    params['location'] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params).json()
    try:
        perf_temp.loc[index,'Hotel Name'] = hotel_name['results'][0]['name']
    except:
        perf_temp.loc[index,'Hotel Name'] = 'No Hotel'

In [12]:
# perf_temp_reduced

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perf_temp.iterrows()]
locations = perf_temp[["Lat", "Lng"]]
# locations

In [14]:
#create the heatmap
hotel_fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(lat_lng,weights=humidity,dissipating=False, max_intensity=100,
#                                  point_radius = 1)
hotel_fig.add_layer(heat_layer)
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
hotel_fig.add_layer(marker_layer)
# Display figure
hotel_fig

Figure(layout=FigureLayout(height='420px'))